In [28]:
import numpy as np 
import pandas as pd 
import os
print(os.listdir("../input"))

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [29]:
df_raw = pd.read_csv('../input/train.csv');df_raw.head(2)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0


In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
df = df_raw.loc[df_raw['target'] == 1]

In [33]:
df.shape

(80810, 3)

In [34]:
df1 = df_raw.loc[df_raw['target'] == 0].loc[:490810]; df1.shape

(460422, 3)

In [35]:
result = pd.concat([df, df1]); result.shape

(541232, 3)

In [36]:
X_trn, X_vld, y_trn, y_vld = train_test_split(result['question_text'], result['target'], test_size=0.0001)

In [37]:
X_trn.shape, X_vld.shape

((541177,), (55,))

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
veczr = CountVectorizer()

In [40]:
trn_term_doc = veczr.fit_transform(X_trn)
val_term_doc = veczr.transform(X_vld)

In [41]:
vocab = veczr.get_feature_names(); vocab[5000:5005]

['92cd', '92nd', '92percentile', '92s', '93']

In [42]:
x= trn_term_doc.sign()
y = y_trn

In [43]:
from sklearn.metrics import f1_score

In [44]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(x, y)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=100000000.0, class_weight=None, dual=True,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [45]:
f1_score(y_vld, m.predict(veczr.transform(X_vld).sign()))

0.6666666666666666

In [46]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y)
f1_score(y_vld, m.predict(veczr.transform(X_vld).sign()))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.75

In [47]:
f1_score(y_vld, m.predict(veczr.transform(X_vld).sign()))

0.75

In [48]:
veczr =  CountVectorizer(ngram_range=(1,4), tokenizer=None)
trn_term_doc = veczr.fit_transform(X_trn)
val_term_doc = veczr.transform(X_vld)

In [49]:
y=y_trn
x=trn_term_doc.sign()
val_x = val_term_doc.sign()
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y);

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [50]:
f1_score(y_vld, m.predict(veczr.transform(X_vld).sign()))

0.75

In [51]:
test_df = pd.read_csv("../input/test.csv"); test_df.head()

,qid,question_text
0,00014894849d00ba98a9,My voice range is A2-C5. My chest voice goes u...
1,000156468431f09b3cae,How much does a tutor earn in Bangalore?
2,000227734433360e1aae,What are the best made pocket knives under $20...
3,0005e06fbe3045bd2a92,Why would they add a hypothetical scenario tha...
4,00068a0f7f41f50fc399,What is the dresscode for Techmahindra freshers?


In [52]:
df_test = test_df['question_text']

In [53]:
preds = m.predict(veczr.transform(df_test).sign())

In [54]:
pred_test_y = preds.astype(int)
test_df = pd.read_csv("../input/test.csv", usecols=["qid"])
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)

In [55]:
out_df.head()

,qid,prediction
0,00014894849d00ba98a9,0
1,000156468431f09b3cae,0
2,000227734433360e1aae,0
3,0005e06fbe3045bd2a92,0
4,00068a0f7f41f50fc399,0
